In [1]:
import torch as tch
import torchvision.datasets as dt
import torchvision.transforms as trans
import torch.nn as nn
import matplotlib.pyplot as plt
from time import time
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

train = dt.MNIST(root="./datasets", train=True, transform=trans.ToTensor(), download=True)
test = dt.MNIST(root="./datasets", train=False, transform=trans.ToTensor(), download=True)
print("No. of Training examples: ",len(train))
print("No. of Test examples: ",len(test))

train_batch = tch.utils.data.DataLoader(train, batch_size=30, shuffle=True)

No. of Training examples:  60000
No. of Test examples:  10000


In [3]:
input = 784
hidden = 490
output = 10

model = nn.Sequential(nn.Linear(input, hidden),
                      nn.LeakyReLU(),
                      nn.Linear(hidden, output),
                      nn.LogSoftmax(dim=1))

lossfn = nn.NLLLoss()
images, labels = next(iter(train_batch))
images = images.view(images.shape[0], -1)

logps = model(images)
loss = lossfn(logps, labels)
loss.backward()

optimize = tch.optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time_start = time()
epochs = 18
for num in range(epochs):
    run=0
    for images, labels in train_batch:
        images = images.view(images.shape[0], -1)
        optimize.zero_grad()
        output = model(images)
        loss = lossfn(output, labels)
        writer.add_scalar("Loss", loss, num)
        loss.backward()
        optimize.step()
        run += loss.item()
    else:
        print("Epoch Number : {} = Loss : {}".format(num, run/len(train_batch)))
Elapsed=(time()-time_start)/60
print("\nTraining Time (in minutes) : ",Elapsed)
writer.flush()
writer.close()

Epoch Number : 0 = Loss : 0.5081581976301968
Epoch Number : 1 = Loss : 0.2607014692854136
Epoch Number : 2 = Loss : 0.20602162138931454
Epoch Number : 3 = Loss : 0.16912483075121418
Epoch Number : 4 = Loss : 0.14195448012137785
Epoch Number : 5 = Loss : 0.12255496634542942
Epoch Number : 6 = Loss : 0.10741885769902729
Epoch Number : 7 = Loss : 0.09534787850477733
Epoch Number : 8 = Loss : 0.08542947270977311
Epoch Number : 9 = Loss : 0.07763797796762083
Epoch Number : 10 = Loss : 0.07077616441179999
Epoch Number : 11 = Loss : 0.06475602737592999
Epoch Number : 12 = Loss : 0.05975847951992182
Epoch Number : 13 = Loss : 0.05511430356022902
Epoch Number : 14 = Loss : 0.05111963468440808
Epoch Number : 15 = Loss : 0.04776831939723342
Epoch Number : 16 = Loss : 0.04453270577394869
Epoch Number : 17 = Loss : 0.0416670137080946

Training Time (in minutes) :  2.8615051031112673


In [4]:
correct=0
all = 0
for images,labels in test:
  img = images.view(1, 784)
  with tch.no_grad():
    logps = model(img)   
  ps = tch.exp(logps)
  probab = list(ps.numpy()[0])
  prediction = probab.index(max(probab))
  truth = labels
  if(truth == prediction):
    correct += 1
  all += 1

print("Number Of Images Tested : ", all)
print("Model Accuracy : ", (correct/all))

Number Of Images Tested :  10000
Model Accuracy :  0.9787
